In [15]:
include("pegasos.jl")
using MLDatasets

In [67]:
using PyCall
docvec = pyimport("gensim.models.doc2vec")
model = docvec[:Doc2Vec][:load]("trained_modals/A-model-en.d2v")[:docvecs]
vec = PyVector(model)


300-element Array{Float32,1}:
  0.208127  
  0.162946  
  0.0466024 
 -0.12001   
 -0.032716  
  0.00707923
  0.0532549 
  0.0274912 
  0.213107  
  0.150207  
  0.0552402 
 -0.194502  
 -0.0394107 
  ⋮         
 -0.141326  
 -0.146654  
  0.107221  
 -0.0831539 
 -0.0951512 
  0.250734  
 -0.235098  
  0.100272  
 -0.0204125 
  0.0387626 
 -0.0513165 
  0.140836  

In [3]:
train_data, train_labels = MNIST.traindata()

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [5, 0, 4, 1, 9, 2, 1, 3, 1, 4  …  9, 2, 9, 5, 1, 8, 3, 5, 6, 8])

In [4]:
#build training vectors
test_vecs = Array{Float64}(0,784)
for i in 1:2000
    test_vecs = vcat(test_vecs, reshape(train_data[:,:,i],1,:))
end
test_vecs


2000×784 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0

In [5]:
#build th
y_unfiltered = train_labels[1:2000]
#create indices for recoginizing 0
zero_indices = filter(x -> train_labels[x] == 0, 1:2000)
y_filtered = zeros(2000,1)
y_filtered[:] = -1.0
y_filtered[zero_indices] = 1.0
y_filtered

2000×1 Array{Float64,2}:
 -1.0
  1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
  ⋮  
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
  1.0
 -1.0
 -1.0
 -1.0
  1.0

In [6]:
function error_rate(test, real)
    trials = size(test,1)
    ninc = 0
    for i in 1:trials
        if test[i] != real[i]
            ninc +=1
        end
    end
    return ninc/trials     
end

error_rate (generic function with 1 method)

In [29]:
#Now basic Thikhonov reg
X = test_vecs
y = y_filtered
lamb = 0.5
w = (X'*X + lamb *eye(784))^(-1)*X'*y
#now use w to make preds
pred_vecs = Array{Float64}(0,784)
for i in 2001:3000
    pred_vecs = vcat(pred_vecs, reshape(train_data[:,:,i],1,:))
end
real_unfiltered = train_labels[2001:3000]
zero_indices = filter(x-> real_unfiltered[x] == 0, 1:1000)
real_ans = real_unfiltered
real_ans[:] = -1.0
real_ans[zero_indices] = 1.0
ans = sign.(pred_vecs*w)
error_rate(ans, real_ans)


0.016

# Now using pegasos to test an SVM method

In [44]:
#define a kernel function
function RBF(x::Array{Float64}, y::Array{Float64})
    return exp(-norm(x - y)/2)
end  


alpha = pegasos(X,y, 0.4, 500, (x,x_1) -> norm(x- x_1)/size(x,1))



784-element Array{Float64,1}:
 0.0
 0.0
 1.0
 0.0
 1.0
 1.0
 1.0
 0.0
 1.0
 1.0
 1.0
 0.0
 0.0
 ⋮  
 0.0
 2.0
 0.0
 2.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0

In [45]:
w = alpha_to_w(alpha, 0.4, 500, X, y, x->x)
preds = sign.(pred_vecs*w)
error_rate(preds, real_ans)

0.094